# Imports

In [1]:
!pip install ultralytics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.5/616.5 kB 12.2 MB/s eta 0:00:00


# Dataset

In [16]:
!pip install roboflow -q

from roboflow import Roboflow
rf = Roboflow(api_key="MS15bAajd3WoxFsALdhF")
project = rf.workspace("innopolis-university-5kqay").project("grocery-products-detection")
dataset = project.version(2).download("yolov8")
print(f"Location: {dataset.location}")

RuntimeError: ignored

## Move data.yaml to the root

In [3]:
!cp Grocery-products-detection-2/data.yaml data.yaml

# Train YOLOv8 small

In [4]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data={dataset.location}/../data.yaml \
epochs=100 \
imgsz=640

100% 21.5M/21.5M [00:00<00:00, 199MB/s]
Ultralytics YOLOv8.0.192 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/Grocery-products-detection-2/../data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

# Validate the trained model

In [9]:
!yolo task=detect \
mode=val \
model=/content/runs/detect/train/weights/best.pt \
data={dataset.location}/../data.yaml

Ultralytics YOLOv8.0.192 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
val: Scanning /content/Grocery-products-detection-2/valid/labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100% 21/21 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/Grocery-products-detection-2/valid/images/Bread4_jpg.rf.1986c951a4ed26bab71d6f42c8fbfab7.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:01<00:00,  1.00it/s]
                   all         21         99      0.895      0.773      0.912      0.784
                Apples         21         12      0.994          1      0.995      0.995
               Bananas         21         31      0.959      0.762       0.92      0.781
                 Bread         21          3      0.968      0.667      0.913      0.504
                Carrot         21         18      0.873      0.444

# Test trained model

In [11]:
!yolo task=detect \
mode=predict \
model=/content/runs/detect/train/weights/best.pt \
conf=0.4 \
source={dataset.location}/test/images

Ultralytics YOLOv8.0.192 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs

image 1/11 /content/Grocery-products-detection-2/test/images/Bananas1_jpg.rf.43657a96d1c4f3d1c9a1d6bf6953d8a5.jpg: 640x480 6 Bananass, 101.6ms
image 2/11 /content/Grocery-products-detection-2/test/images/Bananas7_jpg.rf.01b8b6592c0c392fde85220d8ce7df90.jpg: 640x480 3 Bananass, 1 Cucumber, 1 Tomatoes, 12.1ms
image 3/11 /content/Grocery-products-detection-2/test/images/Bread1_jpg.rf.a2ebabbcfb8a1e98ffdc74b3c9ded1ef.jpg: 640x480 1 Apples, 2 Bananass, 2 Cucumbers, 12.1ms
image 4/11 /content/Grocery-products-detection-2/test/images/Bread6_jpg.rf.9722865773fbda23dac78b02e651f880.jpg: 640x480 1 Apples, 3 Bananass, 1 Bread, 3 Cucumbers, 12.1ms
image 5/11 /content/Grocery-products-detection-2/test/images/Chesse7_jpg.rf.7618d659cfb2c5dd1ba7115da15b1b04.jpg: 640x480 1 Apples, 2 Bananass, 2 Carrots, 1 Cheese, 3 Cucumbers, 12.0ms
i

# Deploy the model to roboflow project

In [15]:
project.version(dataset.version).deploy(model_type='yolov8', model_path=f'/content/runs/detect/train/')


Dependency ultralytics==8.0.134 is required but found version=8.0.192, to fix: `pip install ultralytics==8.0.134`
Would you like to continue with the wrong version of ultralytics? y/n: y
View the status of your deployment at: https://app.roboflow.com/innopolis-university-5kqay/grocery-products-detection/2
Share your model with the world at: https://universe.roboflow.com/innopolis-university-5kqay/grocery-products-detection/model/2
